In [9]:
#Top 6 Popularity without duplicate

import pandas as pd
import matplotlib.pyplot as plt

# 1) Load
df = pd.read_csv("spotify_songs.csv")

# 2) Make sure popularity is numeric and drop NaNs
df['track_popularity'] = pd.to_numeric(df['track_popularity'], errors='coerce')
df = df.dropna(subset=['track_popularity'])

# 3) Build normalized keys to dedupe reliably (ignore case/whitespace)
df['_name_key'] = df['track_name'].str.strip().str.lower()
df['_artist_key'] = df['track_artist'].str.strip().str.lower()

# 4) Keep the row with the highest popularity for each (song, artist)
idx = df.groupby(['_name_key', '_artist_key'])['track_popularity'].idxmax()
df_unique = df.loc[idx].copy()

# 5) Top 6 overall (allow same song name with different artists)
top6 = df_unique.sort_values('track_popularity', ascending=False) \
                .head(6)

print("Top 6 Most Popular Songs (unique by song+artist, highest popularity kept):")
print(top6[['song_id','track_name', 'track_artist', 'track_popularity']])

# 6) Plot
#plt.figure(figsize=(10, 6))
#labels = top6['track_name'] + " - " + top6['track_artist']
#plt.barh(labels, top6['track_popularity'])
#plt.gca().invert_yaxis()  # most popular at top
#plt.xlabel("Popularity")
#plt.title("Top 6 Most Popular Songs (Unique by Song + Artist)")
# annotate values
#for i, v in enumerate(top6['track_popularity']):
#    plt.text(v + 0.5, i, f"{int(v)}", va='center')
#plt.show()


Top 6 Most Popular Songs (unique by song+artist, highest popularity kept):
       song_id    track_name    track_artist  track_popularity
1551   1862594  Dance Monkey     Tones and I               100
20092  4400529       ROXANNE  Arizona Zervas                99
1301   1831474       Circles     Post Malone                98
687      42965          Tusa         KAROL G                98
5508   5050341       The Box     Roddy Ricch                98
711    3044971      Memories        Maroon 5                98


In [2]:
#New Release Songs
import pandas as pd
import matplotlib.pyplot as plt

# =======================
# 1. Load dataset
# =======================
file_path = "spotify_songs.csv"
df = pd.read_csv(file_path)

# =======================
# 2. Parse release date
# =======================
df['track_album_release_date'] = pd.to_datetime(
    df['track_album_release_date'], errors='coerce'
)

# Drop rows with invalid or missing dates
df = df.dropna(subset=['track_album_release_date'])

# =======================
# 3. Remove duplicates (same track + artist, keep latest release)
# =======================
df['_name_key'] = df['track_name'].str.strip().str.lower()
df['_artist_key'] = df['track_artist'].str.strip().str.lower()
idx = df.groupby(['_name_key', '_artist_key'])['track_album_release_date'].idxmax()
df_unique = df.loc[idx].copy()

# =======================
# 4. Get Top N newest releases
# =======================
top_new = df_unique.sort_values('track_album_release_date', ascending=False).head(6)

print("Newest 6 Released Songs (unique by song+artist):")
print(top_new[['track_name', 'track_artist', 'track_album_release_date']])

# =======================
# 5. Plot
# =======================
#plt.figure(figsize=(10, 6))
#labels = top_new['track_name'] + " - " + top_new['track_artist']
#plt.barh(labels, top_new['track_album_release_date'].dt.year, color="green")
#plt.gca().invert_yaxis()  # newest at the top
#plt.xlabel("Release Year")
#plt.title("Newest 6 Released Songs")
#plt.show()


Newest 6 Released Songs (unique by song+artist):
                      track_name      track_artist track_album_release_date
382                          別世界           E-girls               2020-01-29
1198                      Senses          KINGDØMS               2020-01-20
23597              Play the Fool      Gramm Thomas               2020-01-17
28811  It Could Be - Genix Remix  Armin van Buuren               2020-01-17
701          What A Man Gotta Do    Jonas Brothers               2020-01-17
28647                    Therapy       Duke Dumont               2020-01-17


C:\Users\User\AppData\Local\Temp\ipykernel_12140\1759941043.py:14: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['track_album_release_date'] = pd.to_datetime(


In [2]:
# Find by song name (for the search bar UI)

# ====================================
# 1. Import Libraries
# ====================================
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

# ====================================
# 2. Load Dataset
# ====================================
file_path = "spotify_songs.csv"   # make sure it's in the same folder
df = pd.read_csv(file_path)

# ====================================
# 3. Interactive Search Box (Sorted by Popularity, with Song ID first)
# ====================================
def interactive_search(keyword):
    results = df[df['track_name'].str.contains(keyword, case=False, na=False)][[
        'song_id', 'track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'track_popularity'
    ]]
    
    if results.empty:
        print(f"❌ No songs found with keyword '{keyword}'.")
    else:
        # Sort by popularity (highest first)
        results = results.sort_values(by='track_popularity', ascending=False)
        display(results.head(20))  # show top 20

widgets.interact(interactive_search, keyword="")


interactive(children=(Text(value='', description='keyword'), Output()), _dom_classes=('widget-interact',))

<function __main__.interactive_search(keyword)>

In [ ]:
#When user select one song
import pandas as pd

# --- Load song metadata ---
metadata_file = "spotify_songs.csv"
songs_meta = pd.read_csv(metadata_file)

# --- Function to display track info ---
def display_song_info(song_id):
    try:
        # Filter the song by song_id
        song_info = songs_meta[songs_meta["song_id"] == song_id]

        if song_info.empty:
            return f"Song ID {song_id} not found in the dataset."

        # Select required columns
        song_info = song_info[[
            "track_album_name", 
            "track_name", 
            "track_artist", 
            "playlist_genre", 
            "playlist_subgenre", 
            "track_album_release_date"
        ]]

        # Reset index for neat display
        return song_info.reset_index(drop=True)
    
    except Exception as e:
        return f"Error: {str(e)}"

# --- Example Usage ---
user_input_song_id = int(input("Enter the Song ID: "))
print("\nTrack Information:")
print(display_song_info(user_input_song_id))

In [8]:
#User Login

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Load CSV file ---
df_cf = pd.read_csv("piki_filtered.csv")

# Store login state
login_status = {"is_logged_in": False, "user_id": None}

# --- Widgets ---
user_input = widgets.Text(
    placeholder='Enter User ID',
    description='User ID:',
    disabled=False
)

login_button = widgets.Button(
    description="Login",
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to login/logout",
    icon="sign-in"    # (FontAwesome icon)
)

status_label = widgets.Label(value="Status: Not logged in")

# --- Login/Logout function ---
def handle_login(b):
    global login_status
    
    if not login_status["is_logged_in"]:
        entered_id = user_input.value.strip()
        
        # Check if user_id exists in the dataset
        if entered_id in df_cf["user_id"].astype(str).values:
            login_status["is_logged_in"] = True
            login_status["user_id"] = entered_id
            
            # Update UI
            user_input.disabled = True
            login_button.description = "Logout"
            login_button.icon = "sign-out"
            status_label.value = f"✅ Logged in as User {entered_id}"
        else:
            status_label.value = "❌ User ID not found!"
    else:
        # Logout
        login_status["is_logged_in"] = False
        login_status["user_id"] = None
        
        # Reset UI
        user_input.disabled = False
        user_input.value = ""
        login_button.description = "Login"
        login_button.icon = "sign-in"
        status_label.value = "Status: Logged out"

# --- Attach event handler ---
login_button.on_click(handle_login)

# --- Display UI ---
display(user_input,login_button,status_label)

Text(value='', description='User ID:', placeholder='Enter User ID')

Button(description='Login', icon='sign-in', style=ButtonStyle(), tooltip='Click to login/logout')

Label(value='Status: Not logged in')